In [63]:
import numpy as np  
import os 
import cv2 
from scipy.ndimage import convolve
from skimage.exposure import match_histograms 
from skimage.morphology import remove_small_objects, binary_opening

Section below for uploading the images on the system. 

In [64]:
dataset_path = '/home/sg24duk/Desktop/datapython/'

images = []
labels = []


finallypath = sorted((os.listdir(dataset_path)))
i = 0
for filename in finallypath:
    
    if filename.endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')): 
        
        img_path = os.path.join(dataset_path,filename)
        img = cv2.imread(img_path)
        images.append(img)
    

# Convert to numpy arrays
images = np.array(images)
images_algo = np.transpose(images, (1,2,3,0))

print(images_algo.shape)

(384, 512, 3, 30)


Parameters Area 

In [145]:
[s1,s2,s3,s4] = (images_algo.shape) # size of the images

refIdx = 14 ### State the REF pic

r  = 12;                    # slide size of the guided filter
alpha = 1.1;                # the parameters for the last fusion
SigD = 0.12                 # the Gauss parameters in the weigh map of the detail layer fusion
p = 4                       # ?
gSig = 0.2                  # 
lSig = 0.5                  # 
wSize = 21                  # Square Kernel length
stepSize = 2                
exposureThres = 0.01        ##
consistencyThres = 0.15
structureThres = 0.80
overexposure_pa = 0.5
C = 0.03 ** 2 / 2          # inherited from SSIM
epsil = 0.25               # (0.5*1)^2;
np_i = s1*s2            
H = np.ones((7,7))            # small average filter is enough
H = H / H.sum()               # Normalized it. 
Fake_Ref = np.zeros((s1, s2, s3))
L = np.zeros((s1, s2, s4))
gMu = np.zeros((s1, s2, s4))      # global mean intensity
BaseMu = np.zeros((s1, s2, s4))   # local mean intensity

numExd = 2*s4 - 1

xIdxMax = s1 - wSize + 1
yIdxMax = s2 - wSize + 1

window = np.ones((wSize, wSize))
window3D = np.repeat(window[:, :, np.newaxis], 3, axis=2)


lMu_c = np.zeros((xIdxMax, yIdxMax, numExd))  # Local mean intensity
lMuSq_c = np.zeros((xIdxMax, yIdxMax, numExd))  # Local mean intensity squared
sigmaSq = np.zeros((xIdxMax, yIdxMax, numExd))  # Signal strength from variance


Function Define Area 

In [173]:
def im2double(image):
    return image.astype(np.float32) / 255.0   ## Images are converted to double!

def convolve2D(image, kernel, padding=0, strides=1): ## To implement 2-D Convolution operation. 

    # Cross Correlation
    kernel = np.flipud(np.fliplr(kernel))

    # Gather Shapes of Kernel + Image + Padding
    xKernShape = kernel.shape[0]
    yKernShape = kernel.shape[1]
    xImgShape = image.shape[0]
    yImgShape = image.shape[1]

    # Shape of Output Convolution
    xOutput = int(((xImgShape - xKernShape + 2 * padding) / strides) + 1)
    yOutput = int(((yImgShape - yKernShape + 2 * padding) / strides) + 1)
    output = np.zeros((xOutput, yOutput))

    # Apply Equal Padding to All Sides
    if padding != 0:
        imagePadded = np.zeros((image.shape[0] + padding*2, image.shape[1] + padding*2))
        imagePadded[int(padding):int(-1 * padding), int(padding):int(-1 * padding)] = image
        print(imagePadded)
    else:
        imagePadded = image

    # Iterate through image
    for y in range(image.shape[1]):
        # Exit Convolution
        if y > image.shape[1] - yKernShape:
            break
        # Only Convolve if y has gone down by the specified Strides
        if y % strides == 0:
            for x in range(image.shape[0]):
                # Go to next row once kernel is out of bounds
                if x > image.shape[0] - xKernShape:
                    break
                try:
                    # Only Convolve if x has moved by the specified Strides
                    if x % strides == 0:
                        output[x, y] = (kernel * imagePadded[x: x + xKernShape, y: y + yKernShape]).sum()
                except:
                    break

    return output

def imfConsistency(mu, refIdx, consistencyThres): ## Imitation of imfConsistency.m 
    # Get the dimensions of the input 3D array `mu`
    s1, s2, s3 = mu.shape
    
    # Initialize cMap with zeros and set the reference slice to ones
    cMap = np.zeros((s1, s2, s3))
    cMap[:, :, refIdx] = np.ones((s1, s2))
    
    # Reference slice of `mu`
    refMu = mu[:, :, refIdx]
    
    # Histogram matching and comparison for each slice except the reference
    for i in range(s3):
        if i != refIdx:
            # Histogram matching
            cMu = match_histograms(mu[:, :, i], refMu, channel_axis=None)
            
            # Calculate absolute difference and update cMap based on threshold
            diff = np.abs(cMu - refMu)
            cMap[:, :, i] = diff <= consistencyThres  # Binary map based on consistency threshold
    
    return cMap

def rgb2grey(X): ## This is the custom one. 
    orig_shape = X.shape
    threeD = (X.ndim == 3)

    # Define the transformation coefficients (for red, green, blue channels respectively)
    coef = np.array([0.2, 0.6, 0.2])

    if threeD:
        # RGB case
        if X.dtype in [np.float32, np.float64]:
            # Reshape to a 2D array where each row is a pixel, with 3 columns (R, G, B)
            X_reshaped = X.reshape(-1, 3)
            # Apply grayscale transformation
            I = np.dot(X_reshaped, coef)
            # Clip to ensure values are within [0, 1] range for float types
            I = np.clip(I, 0, 1)
            # Reshape back to the original 2D dimensions
            I = I.reshape(orig_shape[0], orig_shape[1])
        
        else:
            # For uint8 or uint16 types, convert by scaling appropriately
            I = np.dot(X, coef).astype(X.dtype)
    else:
        # For 2D input (already grayscale), repeat channels for RGB equivalent output
        I = np.dot(X, coef)
        I = np.clip(I, 0, 1)
        I = np.stack([I, I, I], axis=-1)
    
    return I

def rgb2gray(image): ## that's from MATLAB
    """
    Convert an RGB image to grayscale.
    
    Parameters:
    - image: A 3D NumPy array (H, W, 3) representing an RGB image.

    Returns:
    - A 2D NumPy array (H, W) representing the grayscale image.
    """
    # Ensure the input image is a 3D array
    if image.ndim != 3 or image.shape[2] != 3:
        raise ValueError("Input image must be a 3D array with 3 channels (RGB).")
    
    # Define the coefficients for RGB to grayscale conversion
    coef = np.array([0.2989, 0.5870, 0.1140])
    
    # Convert to grayscale using dot product
    gray_image = np.dot(image[..., :3], coef)
    
    # Clip values to [0, 1] if image is normalized, or [0, 255] if uint8
    if gray_image.max() > 1:
        gray_image = (np.clip(gray_image, 0, 255).astype(np.uint8)) / 255
    else:
        gray_image = np.clip(gray_image, 0, 1)
    
    return gray_image

def imfConsistency(mu, refIdx, consistencyThres):
    # Get the dimensions of the input 3D array `mu`
    s1, s2, s3 = mu.shape
    
    # Initialize cMap with zeros and set the reference slice to ones
    cMap = np.zeros((s1, s2, s3))
    cMap[:, :, refIdx] = np.ones((s1, s2))
    
    # Reference slice of `mu`
    refMu = mu[:, :, refIdx]
    
    # Histogram matching and comparison for each slice except the reference
    for i in range(s3):
        if i != refIdx:
            # Histogram matching
            cMu = match_histograms(mu[:, :, i], refMu, channel_axis=None)
            
            # Calculate absolute difference and update cMap based on threshold
            diff = np.abs(cMu - refMu)
            cMap[:, :, i] = diff <= consistencyThres  # Binary map based on consistency threshold
    
    return cMap






In [181]:
### Convert Every image to double 

imgSeqExtended = np.zeros((s1,s2,s3,numExd))
imgSeq = np.zeros((s1,s2,s3,s4))
imgSeqExtended[:,:,:,:s4] = im2double(images_algo)
imgSeq = im2double(images_algo)

### Initialize temporary values. 
temp = np.zeros((s1,s2,s3))
sq_temp = np.zeros((s1,s2,s3))
## calculate local mean intensity (also might be called luminance) and Squared value of it to calculate sigma squared eventually. 
for i in range(numExd):
    temp = imgSeqExtended[:,:,:,i]
    sq_temp = temp*temp
    temp_avg    = np.mean(temp,axis=2)
    sq_temp_avg = np.mean(sq_temp,axis=2)
    lMu_c[:,:,i] = convolve2D(temp_avg,window,0,1)      ##Convolution with averaging kernel 
    lMuSq_c[:,:,i]= lMu_c[:,:,i] * lMu_c[:,:,i]
    sigmaSq[:,:,i] = convolve2D(sq_temp_avg,window,0,1) - lMuSq_c[:,:,i] ## Calculate the variance 


 

In [182]:
print(lMu_c[0:5,0:5,2])

[[0.01176471 0.01191588 0.01208483 0.01228047 0.01251167]
 [0.01163132 0.01178249 0.01195145 0.01214708 0.01236939]
 [0.01182695 0.01198702 0.01218265 0.01238718 0.01260949]
 [0.01150683 0.01169357 0.01192477 0.01216487 0.01242275]
 [0.01083989 0.01104442 0.01129341 0.01156907 0.01186252]]


In [183]:
##This part takes So much time so leave it seperated 
sigma = np.sqrt(np.maximum(sigmaSq,0)) ## Calculate the variance
average = np.mean(imgSeqExtended,axis=2) ## get the average of color channel for easier version of 2-D convolution
sMap_c = np.zeros((xIdxMax,yIdxMax,s4,s4)) ## Structural consistency initializiation


for i in range(s4):
    for j in range(i+1,s4):                     ##Calculation comes from SSIM Formula (third term) but here it is implemented with a bit different technique
        crossMu = lMu_c[:,:,i] * lMu_c[:,:,j]
        crossSigma = convolve2D((average[:,:,i]*average[:,:,j]),window,0,1)  - crossMu
        sMap_c [:,:,i,j] = (crossSigma + C) / (sigma[:,:,i] * sigma[:,:,j]+C)



In [185]:
print(sMap_c[0,89,2,9])

0.9435083750969269


In [149]:

sMap_c_backup = sMap_c  ## it takes so much time to produce sMap_c so just a measure to override it 
squeezed_map = np.squeeze(sMap_c_backup[:, :, refIdx, :])   ## result of cross comparasion is reordered by refIdx
sRefMap = squeezed_map + sMap_c_backup[:, :, :, refIdx]    
sRefMap[:, :, refIdx] = np.ones((xIdxMax, yIdxMax)) ## And filled with 1
sRefMap[sRefMap <= structureThres] = 0  ## compare and binarize.
sRefMap[sRefMap > structureThres] = 1
muIdxMap = (lMu_c[:, :, refIdx] < exposureThres) | (lMu_c[:, :, refIdx] > 1 - exposureThres)
muIdxMap = np.tile(muIdxMap[:, :, np.newaxis], (1, 1, s4))
sRefMap[muIdxMap] = 1

UP TO NOW, we are aligning with MATLAB (tested)

In [186]:

iRefMap = imfConsistency(lMu_c[:,:,:s4], refIdx, consistencyThres)  ##match_histograms function is creating a problem resulting in different results with matlab.

cMap = sRefMap * iRefMap
cMap = np.clip(cMap, 0, 1)
RefMatrix = np.ones_like(cMap)



In [187]:
cMap = sRefMap * iRefMap
cMap = np.clip(cMap, 0, 1)
RefMatrix = np.ones_like(cMap)
t_ref = 0.01
Ig1 = rgb2gray(imgSeqExtended[:, :, :, 14])
Ig1[Ig1 > (1 - (t_ref * 2))] = 1
Ig1[Ig1 < (t_ref * 2)] = 1
Ig1[Ig1 < 1] = 0

In [189]:
m1, m2, m3 = cMap.shape
cMap_seq = np.zeros((m1 + wSize - 1, m2 + wSize - 1, m3))

for icmp in range(m3):
    # Resize the current cMap slice
    resized_cMap = cv2.resize(cMap[:, :, icmp], (m2 + wSize - 1, m1 + wSize - 1))
    # Add Ig1 to the resized map
    cMap_seq[:, :, icmp] = resized_cMap + Ig1
# Set all positive values to 1
cMap_seq[cMap_seq > 0] = 1
# Set all negative values to 0
cMap_seq[cMap_seq < 0] = 0



In [190]:

m1,m2,m3 = cMap_seq.shape
ratio = 0.001
area = int(np.ceil(ratio * m1 * m2))
for i in range(m3):
        # Remove small objects based on the calculated area
        tempMap1 = remove_small_objects(cMap_seq[:, :, i].astype(bool), min_size=area, connectivity=2)

        # Invert the binary image
        tempMap2 = ~tempMap1

        # Remove small objects from the inverted image
        tempMap3 = remove_small_objects(tempMap2.astype(bool), min_size=area, connectivity=2)

        # Finalize the cleaned mask
        cMap_seq[:, :, i] = ~tempMap3  # Invert back to original binary format
        
s1, s2, s3 = cMap_seq.shape  # Get the shape of the input array
cMap_seq_3D = np.ones((s1, s2, 3, s4), dtype=cMap_seq.dtype)  # Initialize 4D array with ones
for i in range(3):
    cMap_seq_3D[:, :, i, :] = cMap_seq

In [191]:

s4=30
count = 0

for i in range(s4):
        if i != refIdx:
            count += 1
            
            # Perform histogram matching
            ref_image = imgSeqExtended[:, :, :, refIdx]
            target_image = imgSeqExtended[:, :, :, i]

            # Histogram matching using skimage's match_histograms function
            matched_image = match_histograms(target_image, ref_image, channel_axis=2)

            # Clip values to ensure they are in the range [0, 1]
            matched_image = np.clip(matched_image, 0, 1)
            imgSeqExtended[:,:,:,s4+count-1] = matched_image
            # Append the matched image to imgSeqColorExd




In [197]:
count = 0
m1, m2, m3, s4 = images_algo.shape

imgSeq = np.zeros((m1,m2,m3,s4))
imgSeq = im2double(images_algo)

num_diff = np.count_nonzero(RefMatrix-cMap)
print(num_diff)
if num_diff > m1 * m2 * m3 * consistencyThres:  # Adjusted to check against the first slice
    for i in range(s4):
        if i != refIdx:
            count += 1
            # Update imgSeqColor using masking
            imgSeq[:,:,:,i] = (imgSeq[:,:,:,i] * cMap_seq_3D[:,:,:,i] + \
                                   (imgSeqExtended[:,:,:, s4 + count - 1] * np.logical_not(cMap_seq_3D[:,:,:, i])))

5279929


Now for image decomposition line 209 in the commented matlab code in git. 

In [200]:

## get luminance again? 

for i in range(s4):
    
    Igrey = rgb2grey(imgSeq[:,:,:,i])
    IgPad = padimage

In [ ]:
def symmetric_pad(a_size, pad_size, direction):
    num_dims = len(pad_size)
    idx = [None] * num_dims  # Create a list to hold indices for each dimension

    for k in range(num_dims):
        M = a_size[k]
        dim_nums = np.concatenate((np.arange(1, M + 1), np.arange(M, 0, -1))).astype(np.uint32)
        p = pad_size[k]
        idx[k] = dim_nums[np.mod(np.arange(-p, M + p), 2 * M)]
        
    return idx